# US Golf Course Information Kiosk

Detailed information on golf courses with green fees range and tees, this complete golf courses database has 15,606 records across 7,891 cities over 53 states in the United States. Each record is comprised of address, street, phone number, zip code, hole, architect, year built, public/private, guest policy, credit card, golf season, range, rental club, pro in house, metal spikes okay, weekday, weekend, tee time welcomed, rental cart available, championship par/yards/slope/USGA, middle par/yards/slope/USGA and forward par/yards/slope/USGA

https://www.usabledatabases.com/database/golf-courses-in-us/

## Step 1:  Clean and transform the input dataset

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine
#from config import db_connection

# Import API key
from config import gkey

# CSV files
course = "data/course.csv"
city = "data/city.csv"
state = "data/state.csv"

In [2]:
# Create the data frame - golf courses
course_df = pd.read_csv(course, 
                        delimiter=',', 
                        skipinitialspace=True, 
                        dtype={"zip_code":object, "hole":object })

# Add columns for longitude and latitude
course_df["lng"] = ""
course_df["lat"] = ""

golf_courses = course_df.rename(columns={"id": "course_id",
                                         "title": "course"})

# Replace &amp; with &
golf_courses = golf_courses.replace('&amp;','&', regex=True)

golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
0,1,1,Albertville Golf & Country Club,Country Club Rd,"Albertville, Alabama 35950",35950,(256) 878-4403,18,Leon Howard,1966,...,72,6068,117,68.00,72,5196,NaN,NaN,,
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,,
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,,
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,,
4,5,4,Maple Hills Golf,Hwy 75,"Altoona, Alabama 35952",35952,(205) 466-7600,9,Bill Ellison,1975,...,31,1800,NaN,NaN,31,1800,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,15750,8219,Legion Town & Country Club,141 Airport Rd,"Thermopolis, Wyoming 82443",82443,(307) 864-5294,9,Don Brunk,1962,...,35,2966,111,33.90,37,2836,111,35.00,,
15602,15751,8220,Cottonwood Country Club,15th St,"Torrington, Wyoming 82240",82240,(307) 532-3868,18,NaN,NaN,...,72,6298,112,67.70,73,5344,112,69.20,,
15603,15752,8221,Cedar Pines Golf Course,2579 N Hwy 116,"Upton, Wyoming 82730",82730,(307) 468-2847,9,NaN,1984,...,36,3198,NaN,NaN,38,2521,NaN,NaN,,
15604,15753,8222,Wheatland Golf Club,1253 E Cole,"Wheatland, Wyoming 82201",82201,(307) 322-3675,9,NaN,1960,...,36,3064,119,33.95,36,2557,116,34.25,,


In [3]:
# Exclude all golf courses that do not have championship, middle, and forward tee information
nan_value = float("NaN")
golf_courses.replace("", nan_value, inplace=True)
golf_courses.dropna(subset = ["championship_par"], inplace=True)
golf_courses.dropna(subset = ["middle_par"], inplace=True)
golf_courses.dropna(subset = ["forward_par"], inplace=True)
golf_courses.dropna(subset = ["championship_slope"], inplace=True)
golf_courses.dropna(subset = ["middle_slope"], inplace=True)
golf_courses.dropna(subset = ["forward_slope"], inplace=True)
golf_courses.dropna(subset = ["championship_usga"], inplace=True)
golf_courses.dropna(subset = ["middle_usga"], inplace=True)
golf_courses.dropna(subset = ["forward_usga"], inplace=True)

# Exclude all golf courses that do not have hole, public-private, season information
golf_courses.dropna(subset = ["hole"], inplace=True)
golf_courses.dropna(subset = ["public_private"], inplace=True)
golf_courses.dropna(subset = ["golf_season"], inplace=True)

golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,NaN,NaN
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,NaN,NaN
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,NaN,NaN
5,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,72,6151,124,69.00,72,5563,119,71.20,NaN,NaN
8,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,70,5924,118,68.70,72,5086,109,68.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15591,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,72,6741,118,71.00,73,5616,115,73.10,NaN,NaN
15592,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,72,6649,114,69.80,72,5410,110,67.60,NaN,NaN
15594,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,72,6469,122,70.00,72,5878,122,71.90,NaN,NaN
15595,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,72,6404,111,69.10,72,5180,111,68.00,NaN,NaN


In [4]:
# Checking for duplicate rows in golf courses
duplicateRow = golf_courses[golf_courses.duplicated()]
duplicateRow

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat


In [5]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "56001") & (golf_courses["zip_code"] <= "56007"))]

subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
7589,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,72,5970,120,69.30,72,5404,126,71.40,NaN,NaN
7846,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,71,5990,128,69.20,71,5009,126,68.90,NaN,NaN
7848,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,36,3093,114,34.50,36,2464,110,34.00,NaN,NaN
7909,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,72,5624,113,67.00,72,4659,114,67.00,NaN,NaN


In [ ]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "55001") & (golf_courses["zip_code"] <= "56763"))]

subset_df

In [ ]:
# Create a subset data frame - variety of golf courses
bool_series = golf_courses["zip_code"].str.startswith("010", na = False) 
df1 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("6000", na = False) 
df2 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("5600", na = False) 
df3 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("920", na = False) 
df4 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("2001", na = False) 
df5 = golf_courses[bool_series]

subset_df = df1.append(df2, ignore_index = True)
subset_df = subset_df.append(df3, ignore_index = True)
subset_df = subset_df.append(df4, ignore_index = True)
subset_df = subset_df.append(df5, ignore_index = True)
subset_df

In [ ]:
# Create a subset data frame - All golf courses
subset_df = golf_courses

subset_df

In [6]:
# Create the data frame - golf cities
city_df = pd.read_csv(city, delimiter=',', skipinitialspace=True)

golf_cities = city_df.rename(columns={"id": "city_id", 
                                      "title": "city_title", 
                                      "count": "city_count", 
                                      "slug": "city_slug"})
golf_cities

,city_id,state_id,city_title,city_count,city_slug
0,1,1,Albertville,2,albertville
1,2,1,Alexander City,1,alexandercity
2,3,1,Alpine,1,alpine
3,4,1,Altoona,1,altoona
4,5,1,Andalusia,2,andalusia
...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis
7887,8220,53,Torrington,1,torrington
7888,8221,53,Upton,1,upton
7889,8222,53,Wheatland,1,wheatland


In [7]:
# Checking for duplicate rows in golf cities
duplicateRow = golf_cities[golf_cities.duplicated()]
duplicateRow

,city_id,state_id,city_title,city_count,city_slug


In [8]:
# Create the data frame - golf states
state_df = pd.read_csv(state, delimiter=',', skipinitialspace=True)

golf_states = state_df.rename(columns={"id": "state_id", 
                                       "title": "state", 
                                       "count": "state_count", 
                                       "slug": "state_slug"})
golf_states

,state_id,state,state_count,state_slug
0,1,Alabama,272,alabama
1,2,Alaska,16,alaska
2,3,Arizona,308,arizona
3,4,Arkansas,179,arkansas
4,5,California,976,california
5,6,Colorado,223,colorado
6,7,Connecticut,177,connecticut
7,8,Delaware,39,delaware
8,9,District of Columbia,6,districtofcolumbia
9,10,Florida,1094,florida


In [9]:
# Checking for duplicate rows in golf states
duplicateRow = golf_states[golf_states.duplicated()]
duplicateRow

,state_id,state,state_count,state_slug


### Merge the data frames to create one complete dataset

In [10]:
# Merge the city and state dataframes using a left join
city_state_df = pd.merge(golf_cities, golf_states, on="state_id", how="left")
city_state_df

,city_id,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,1,1,Albertville,2,albertville,Alabama,272,alabama
1,2,1,Alexander City,1,alexandercity,Alabama,272,alabama
2,3,1,Alpine,1,alpine,Alabama,272,alabama
3,4,1,Altoona,1,altoona,Alabama,272,alabama
4,5,1,Andalusia,2,andalusia,Alabama,272,alabama
...,...,...,...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis,Wyoming,48,wyoming
7887,8220,53,Torrington,1,torrington,Wyoming,48,wyoming
7888,8221,53,Upton,1,upton,Wyoming,48,wyoming
7889,8222,53,Wheatland,1,wheatland,Wyoming,48,wyoming


In [11]:
# Merge the city_state dataframe with the golf course dataframe using a left join
golf_citystate = pd.merge(subset_df, city_state_df, on="city_id" ,how="left")
golf_citystate

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,forward_usga,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,71.40,NaN,NaN,24,Albert Lea,2,albertlea,Minnesota,484,minnesota
1,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,68.90,NaN,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota
2,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,34.00,NaN,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota
3,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,67.00,NaN,NaN,24,North \n Mankato,1,northmankato,Minnesota,484,minnesota


In [12]:
regex = r'(?P<c_city>[^,]+)\s*,\s*'
golf_address = golf_citystate.join(golf_citystate["address"].str.extract(regex))
golf_address

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city
0,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,NaN,NaN,24,Albert Lea,2,albertlea,Minnesota,484,minnesota,Albert Lea
1,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,NaN,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato
2,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,NaN,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato
3,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,NaN,NaN,24,North \n Mankato,1,northmankato,Minnesota,484,minnesota,North Mankato


In [13]:
# Finding the best city to use
golf_address["city"] = golf_address.c_city.combine_first(golf_address.city_title)

golf_df = golf_address
golf_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city,city
0,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,NaN,24,Albert Lea,2,albertlea,Minnesota,484,minnesota,Albert Lea,Albert Lea
1,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato,Mankato
2,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato,Mankato
3,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,NaN,24,North \n Mankato,1,northmankato,Minnesota,484,minnesota,North Mankato,North Mankato


In [14]:
# Golf courses along the Mississippi River - 
# Minnesota, Wisconsin, Iowa, Illinois, Missouri, Kentucky, Tennessee, Arkansas, Mississippi, Louisiana
mask = golf_df["state_id"].isin([24, 52, 16, 14, 26, 18, 44, 4, 25, 19])
MissRiver_golf = golf_df[mask]
MissRiver_golf

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city,city
0,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,NaN,24,Albert Lea,2,albertlea,Minnesota,484,minnesota,Albert Lea,Albert Lea
1,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato,Mankato
2,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,NaN,24,Mankato,4,mankato,Minnesota,484,minnesota,Mankato,Mankato
3,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,NaN,24,North \n Mankato,1,northmankato,Minnesota,484,minnesota,North Mankato,North Mankato


In [15]:
# Select the golf course columns needed for course longitude, latitude
golf_course_df = MissRiver_golf[["course_id",
                                 "course",
                                 "city",
                                 "state",
                                 "street",
                                 "zip_code",
                                 "lng",
                                 "lat",
                                 "hole",
                                 "public_private",
                                 "golf_season"]]
                     
# Display the new dataframe
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,NaN,NaN,18,Public,year round
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,NaN,NaN,18,Private,Open: 4/01 Closed: 11/01
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,NaN,NaN,9,Public,Open: 4/01 Closed: 11/15
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,NaN,NaN,18,Public,Open: 4/01 Closed: 10/31


In [16]:
# Select the golf course columns needed for golf kiosk - general information
course_info = golf_df[["course_id",
                       "phone",
                       "architect",
                       "year_built",
                       "guest_policy",
                       "credit_card"]]
                     
# Display the new dataframe
course_info

,course_id,phone,architect,year_built,guest_policy,credit_card
0,7676,(507) 373-1061,NaN,NaN,Open,"VISA, MasterCard Welcomed"
1,7937,(507) 387-5636,William Langford & Theodore J. Moreau,1925,Closed,None
2,7939,(507) 625-7665,NaN,1973,Open,"VISA, MasterCard Welcomed"
3,8001,(507) 947-3355,Patrick Wyss,1991,Open,"VISA, MasterCard Welcomed"


In [17]:
# Select the golf course columns needed for golf kiosk - course details
course_details = golf_df[["course_id",
                          "range",
                          "rental_club",
                          "pro_in_House",
                          "metal_spikes_okay",
                          "weekday",
                          "weekend",
                          "tee_time_welcomed",
                          "rental_cart_available"]]
                     
# Display the new dataframe
course_details

,course_id,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,7676,No,Yes,Yes,No,$,$,Yes,Yes
1,7937,Yes,Yes,Yes,No,$,$,Yes,Yes
2,7939,Yes,Yes,Yes,No,$,$,Yes,Yes
3,8001,Yes,Yes,Yes,No,$,$,Yes,Yes


In [18]:
# Select the golf course columns needed for golf kiosk - championship tees
championship_tees = golf_df[["course_id", 
                            "championship_par", 
                            "championship_yards", 
                            "championship_slope", 
                            "championship_usga"]]
                     
# Display the new dataframe
championship_tees

,course_id,championship_par,championship_yards,championship_slope,championship_usga
0,7676,72,6166,122,70.20
1,7937,71,6215,130,70.20
2,7939,36,3304,118,35.50
3,8001,72,6073,117,69.10


In [19]:
# Select the golf course columns needed for golf kiosk - middle tees
middle_tees = golf_df[["course_id",
                       "middle_par",
                       "middle_yards",
                       "middle_slope",
                       "middle_usga"]]
                     
# Display the new dataframe
middle_tees

,course_id,middle_par,middle_yards,middle_slope,middle_usga
0,7676,72,5970,120,69.30
1,7937,71,5990,128,69.20
2,7939,36,3093,114,34.50
3,8001,72,5624,113,67.00


In [20]:
# Select the golf course columns needed for golf kiosk - forward tees
forward_tees = golf_df[["course_id",
                        "forward_par",
                        "forward_yards",
                        "forward_slope",
                        "forward_usga"]]
                     
# Display the new dataframe
forward_tees

,course_id,forward_par,forward_yards,forward_slope,forward_usga
0,7676,72,5404,126,71.40
1,7937,71,5009,126,68.90
2,7939,36,2464,110,34.00
3,8001,72,4659,114,67.00


In [21]:
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,NaN,NaN,18,Public,year round
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,NaN,NaN,18,Private,Open: 4/01 Closed: 11/01
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,NaN,NaN,9,Public,Open: 4/01 Closed: 11/15
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,NaN,NaN,18,Public,Open: 4/01 Closed: 10/31


In [22]:
# Getting the longitude and latitude for each golf course
count = 0

# Use the street, city, state to get lng and lat
for index, row in golf_course_df.iterrows():
    # get street, city, state from golf_course_df
    street = row["street"]
    city = row["city"]
    state = row["state"]
    postal_code = row["zip_code"]
    
    # Getting lat/lng
    query_address_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={street},+{city},+{state}&key={gkey}"
    # print(f"url = {query_address_url}")
        
    query_zip_url = f"https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{postal_code}&key={gkey}"
    # print(f"url = {query_zip_url}")

    response = requests.get(query_address_url).json()
    # print(f"response = {response}")
    
    count = count + 1
    # print(f"#{count} - index = {index}")
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        golf_course_df.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        response_zip = requests.get(query_zip_url).json()
        # print(f"response_zip = {response_zip}")golf_course_df.loc[index, "lat"] = response_zip["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response_zip["results"][0]["geometry"]["location"]["lng"]
        print("Using zip code for lng, lat.")
        # print("Missing field/result... skipping.")

C:\Users\Public\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
# Preview course subset
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,year round
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,Open: 4/01 Closed: 11/01
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,Open: 4/01 Closed: 11/15
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,Open: 4/01 Closed: 10/31


In [24]:
# Create a dataframe to export
golf_courses = golf_course_df[["course_id",
                               "course",
                               "city",
                               "state",
                               "street",
                               "zip_code",
                               "lng",
                               "lat",
                               "hole",
                               "public_private", 
                               "golf_season"]]
                     
# Display the new dataframe
golf_courses

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,year round
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,Open: 4/01 Closed: 11/01
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,Open: 4/01 Closed: 11/15
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,Open: 4/01 Closed: 10/31


In [25]:
# Golf season begin and end months
for index, row in golf_courses.iterrows():
    str_open = row["golf_season"][0:5]
    beg_mnth = row["golf_season"][6:7]
    str_closed = row["golf_season"][11:18]
    end_mnth = row["golf_season"][19:21]
    end_day = row["golf_season"][22:24]

    if str_closed == " Closed" :
        beg_mnth = row["golf_courses"][6:8]
        str_closed = row["golf_courses"][12:19]
        end_mnth = row["golf_courses"][20:22]
        end_day = row["golf_courses"][22:24]
   
    if str_closed == "Closed:" :
        if end_day == "01":
            if end_mnth == "1/":
                end_mnth = "12"
            elif end_mnth == "4/":
                end_mnth = "3"
            elif end_mnth == "5/":
                end_mnth = "4"
            elif end_mnth == "6/":
                end_mnth = "5"
            elif end_mnth == "7/":
                end_mnth = "6"
            elif end_mnth == "8/":
                end_mnth = "7"
            elif end_mnth == "9/":
                end_mnth = "7"          
            elif end_mnth == "10":
                end_mnth = "9"
            elif end_mnth == "11":
                end_mnth = "10"             
            elif end_mnth == "12":
                end_mnth = "11"
        else:
            if end_mnth == "1/":
                end_mnth = "01"
            elif end_mnth == "4/":
                end_mnth = "04"
            elif end_mnth == "5/":
                end_mnth = "05"
            elif end_mnth == "6/":
                end_mnth = "06"
            elif end_mnth == "7/":
                end_mnth = "07"
            elif end_mnth == "8/":
                end_mnth = "08"
            elif end_mnth == "9/":
                end_mnth = "09"          
    else:
        end_mnth = "12"

   
    if str_open == "Open:" :
        if beg_mnth not in ["2", "3", "4", "5", "6", "7", "10", "11", "12"]:
            beg_mnth = "1"
    else:
        beg_mnth = "1"      

    golf_courses.loc[index,"beg_mnth"] = beg_mnth
    golf_courses.loc[index,"end_mnth"] = end_mnth
    
golf_courses

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season,beg_mnth,end_mnth
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,year round,1,12
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,Open: 4/01 Closed: 11/01,4,10
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,Open: 4/01 Closed: 11/15,4,11
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,Open: 4/01 Closed: 10/31,4,10


In [26]:
# Merge the tee information with the golf courses dataframe using a left join
golf_tees1 = pd.merge(golf_courses, championship_tees, on="course_id" ,how="left")
golf_tees2 = pd.merge(golf_tees1, middle_tees, on="course_id" ,how="left")
golf_tees3 = pd.merge(golf_tees2, forward_tees, on="course_id" ,how="left")
golf_tees3

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,championship_slope,championship_usga,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,...,122,70.20,72,5970,120,69.30,72,5404,126,71.40
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,...,130,70.20,71,5990,128,69.20,71,5009,126,68.90
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,...,118,35.50,36,3093,114,34.50,36,2464,110,34.00
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,...,117,69.10,72,5624,113,67.00,72,4659,114,67.00


In [ ]:
# Export the golf course data to csv
golf_tees3.to_csv("results/MissRiver_golf.csv", index=False, header=True)

In [ ]:
# Export the golf course data to csv
golf_tees3.to_csv("results/course_subset.csv", index=False, header=True)

In [27]:
# Merge more details with the golf courses dataframe using a left join
golf_info = pd.merge(golf_tees3, course_info, on="course_id" ,how="left")
golf_info

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,phone,architect,year_built,guest_policy,credit_card
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,...,69.30,72,5404,126,71.40,(507) 373-1061,NaN,NaN,Open,"VISA, MasterCard Welcomed"
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,...,69.20,71,5009,126,68.90,(507) 387-5636,William Langford & Theodore J. Moreau,1925,Closed,None
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,...,34.50,36,2464,110,34.00,(507) 625-7665,NaN,1973,Open,"VISA, MasterCard Welcomed"
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,...,67.00,72,4659,114,67.00,(507) 947-3355,Patrick Wyss,1991,Open,"VISA, MasterCard Welcomed"


In [28]:
# Merge more details with the golf courses dataframe using a left join
golf_details = pd.merge(golf_info, course_details, on="course_id" ,how="left")
golf_details

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,guest_policy,credit_card,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,7676,Green Lea Golf Course,Albert Lea,Minnesota,101 E Richway Dr,56007,-93.368199,43.664333,18,Public,...,Open,"VISA, MasterCard Welcomed",No,Yes,Yes,No,$,$,Yes,Yes
1,7937,Mankato Golf Club,Mankato,Minnesota,Golf Club Rd,56002,-93.977669,44.192062,18,Private,...,Closed,None,Yes,Yes,Yes,No,$,$,Yes,Yes
2,7939,Regulation at Terrace View Golf Course,Mankato,Minnesota,Hwy 22 S,56001,-93.949200,44.163524,9,Public,...,Open,"VISA, MasterCard Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
3,8001,North Links Golf Course,North Mankato,Minnesota,RFD 2,56003,-94.033845,44.173300,18,Public,...,Open,"VISA, MasterCard Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes


In [ ]:
# Export the golf course data to csv
golf_details.to_csv("results/golf_details.csv", index=False, header=True)

In [ ]:
# Export the golf course data to csv
golf_details.to_csv("results/MissRiver_golf_details.csv", index=False, header=True)